# The Relationship Between Tree Diameter and Growth Environment

## Introduction

This project involves using the diameter of a common tree species in Vancouver, compared between different streets.

#### Background Information

The diameter of a tree’s trunk reveals the growth of a tree and is closely relative to the surrounding environment. The growth of the trunk's diameter usually happens after the primary growth; where it's responsible for the height and the depth of its root (Cavallo-Medve, et al., 1971). In addition, Berlyn (2022) states that “Tree height is closely associated with the quality of the site on which the tree is growing (i.e., the climate, soil topography, and biota)". As a result, we are interested in comparing the same kind of tree (Magnolia) on different streets, aiming to observe whether there exists a potential environmental influence on the trees.

#### Our Inferential Research Question

For our inferential research question, our group decided to focus on the following question: "Do the Magnolia trees on Charles street differ in diameter from the diameters of the Magnolia trees on Churchill street? We plan on collecting data from the City of Vancouver Open Data Portal, where we're focusing on the variables of GENUS_NAME, DIAMETER, and STD_STREET. In addition, we plan on having  our population of interest be Magnolia trees on Charles St. and Churchill St (these are streets that some of our group members reside on). For our point estimate, we are doing the mean difference between Magnolia trees’ diameters on Charles St. and Churchill St. For our hypothesis, we have...

Null hypothesis ($H_0$): there is no difference in the trees’ diameter when planting in different streets

Alternate hypothesis ($H_a$): trees’ diameter will be affected when planting in different streets

$H_0\text{: }\mu_1 = \mu_2$

$H_a\text{: }\mu_1 \neq \mu_2$


## Preliminary Results

In [ ]:
library(tidyverse)
library(cowplot)
library(infer)
library(repr)

options(jupyter.plot_mimetypes = c("text/plain", "image/png" ))

The data is loaded from a URL stored in the `url` variable. We mirrored the dataset to our git repository for faster, easier access, organization, and reproducibility. The original is available at [this link](https://opendata.vancouver.ca/explore/dataset/street-trees/download/?format=csv&timezone=America/Los_Angeles&lang=en&use_labels_for_header=true&csv_separator=%3B).

In [ ]:
url <- "https://github.com/armanmoztar/stat-201-group7/raw/main/data.csv"

The data is read and filtered to include only the relevant columns and populations.

In [ ]:
magnolia_trees <- read_delim(url, delim = ";") |>
    filter(GENUS_NAME == "MAGNOLIA") |>
    select(street = STD_STREET, diameter = DIAMETER) |>
    filter(street %in% c("CHARLES ST", "CHURCHILL ST"))

head(magnolia_trees)

We estimate our population parameter from our sample and present it in a table for each street.

In [ ]:
magnolia_tree_means <- magnolia_trees |>
    group_by(street) |>
    summarise(mean = mean(diameter))

magnolia_tree_means

The data is presented as two histograms comparing the diameter of the trees between the two streets, as well as the mean represented by the red line.

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)

charles_st_trees_graph <- magnolia_trees |>
    filter(street == "CHARLES ST") |>
    ggplot(aes(x = diameter)) +
    geom_histogram(bins = 50) +
    geom_vline(xintercept = filter(magnolia_tree_means, street == "CHARLES ST")$mean, color = "red") +
    xlim(0, 35) +
    ylim(0, 80) +
    ggtitle("Magnolia Trees on Charles St.") +
    labs(x = "Diameter (inches)", y = "Count")

churchill_st_trees_graph <- magnolia_trees |>
    filter(street == "CHURCHILL ST") |>
    ggplot(aes(x = diameter)) +
    geom_histogram(bins = 50) +
    geom_vline(xintercept = filter(magnolia_tree_means, street == "CHURCHILL ST")$mean, color = "red") +
    xlim(0, 35) +
    ylim(0, 80) +
    ggtitle("Magnolia Trees on Churchill St.") +
    labs(x = "Diameter (inches)", y = "Count")

plot_grid(charles_st_trees_graph, churchill_st_trees_graph)

options(repr.plot.width = 8, repr.plot.height = 8)

box_plot <- magnolia_trees |>
    ggplot(aes(x = street, y = diameter)) +
    geom_boxplot() +
    ggtitle("Magnolia Tree Diameters") +
    labs(x = "Street", y = "Diameter (inches)")

box_plot

In [ ]:
obs_diff_means <- magnolia_trees |>
    specify(formula = diameter ~ street) |>
    calculate(stat = "diff in means", order = c("CHARLES ST", "CHURCHILL ST"))

In [ ]:
null_diff_means <- magnolia_trees |>
    specify(formula = diameter ~ street) |>
    hypothesise(null = "independence") |>
    generate(type = "permute", reps = 10000) |>
    calculate(stat = "diff in means", order = c("CHARLES ST", "CHURCHILL ST"))

In [ ]:
diff_means_graph <- null_diff_means |>
    visualize() +
    shade_p_value(obs_stat = obs_diff_means, direction = "both") +
    ggtitle("Null Distribution with P-value") +
    xlab("Difference in Means")

diff_means_graph

In [ ]:
means_p_value <- null_diff_means |>
    get_p_value(obs_stat = obs_diff_means, direction = "both") |>
    pull()

means_p_value

In [ ]:
bootstrap_diff_means <- magnolia_trees |>
    specify(formula = diameter ~ street) |>
    generate(type = "bootstrap", reps = 10000) |>
    calculate(stat = "diff in means", order = c("CHARLES ST", "CHURCHILL ST"))

In [ ]:
ci_diff_means <- bootstrap_diff_means |>
    get_ci(level = 0.98, type = "percentile")

ci_diff_means

In [ ]:
bootstrap_graph <- bootstrap_diff_means |>
    visualise() +
    shade_ci(endpoints = ci_diff_means) +
    ggtitle("Bootstrap Distribution with 98% Confidence Interval") +
    xlab("Difference in Means")

bootstrap_graph

## Methods: Plan

#### Test both difference in means AND difference in medians, so that we have 2 location parameters

This report can be used by city officials to determine if there are reasons that affect tree growth based on its diameter. From the data, we can see if there are ways to make tree growth more consistent (determining if there is enough care being taken to keep the environment green, etc...); however, our plot and estimates are not enough to give to stakeholders, as it's important that we must gather more data, perhaps comparing other streets in Vancouver or using different variables such as neighbourhoods.

---
We also plan on doing a 5% significance level (two-tailed hypothesis test), and by using the null hypothesis test to compare the diameter of the Magnolia trees growing on Charles street and Churchill street.

#### Test Statistic

$$
\begin{align*}
\text{Test Statistic} & = \frac{\overline{\mu_1} - \overline{\mu_2}}{\sqrt{\frac{{s_1}^2}{n_1} + \frac{{s_2}^2}{n_2}}}
\\
\text{df} & = \frac{(\frac{{s_1}^2}{n_1} + \frac{{s_2}^2}{n_2})^2}{\frac{{s_1}^4}{{n_1}^2 (n_1 - 1)} + \frac{{s_2}^4}{{n_2}^2 (n_2 - 1)}}
\end{align*}
$$


#### Expectations

We expect to find differences between the two locations, since the distance between the two streets is over 10 km apart, and different streets may have different environments for the growth of the trees. In addition, there are many external factors such as weather (like heavy rain or dry conditions) that can cause different results.

#### Potential Drawbacks

#### Mention that trees may not have been planted at the same time, because it could influence results

We are comparing the same kind of trees (Magnolia) in two different streets, thus, it can also be considered as a paired (dependent) two-sample t-test. However, because the individual trees' growth are independent of each other, we have decided to use the independent two-sample t-test here.

#### Reproducible Analysis

All the codes used in this research can be reproduced to study another kind of tree or other streets. In addition, the dataset offers other variables such as the neighbourhood name that can be used to perform other types of inferential analysis. For future questions, we can try to determine if drought and other conditions affect the growth of trees in Vancouver.

## References

Cavallo-Medved, D., Fenton, M., Milson, B., Riskin, S., Wilson, K. (1971). Life Matters: Connecting Biology to Your World (1st ed., chapter 17). Tophatmonocle Corp.

City of Vancouver (2022, November 3). Street trees. City of Vancouver, Vancouver Board of Parks and Recreation. https://opendata.vancouver.ca/explore/embed/dataset/street-trees/table/?disjunctive.species_name&disjunctive.common_name&disjunctive.on_street&disjunctive.neighbourhood_name&location=11,49.24765,-123.12046

Everett, T. H., Berlyn, . Graeme Pierce and Weber, . Lillian M. (2022, September 21). tree. Encyclopedia Britannica. https://www.britannica.com/plant/tree